In [3]:
import pandas as pd

In [3]:
contributions = pd.read_csv('individual_contributions.csv', sep='|', header=None,
                            usecols=[0, 3, 10, 13, 14, 20],
                            names=['CMTE_ID',
                                  'TRANSACTION_PGI',
                                  'ZIP_CODE',
                                  'TRANSACTION_DT',
                                  'TRANSACTION_AMT',
                                  'SUB_ID'],
                            dtype={"CMTE_ID": str, 
                                   "TRANSACTION_PGI": str,
                                   "ZIP_CODE": str,
                                  'TRANSACTION_DT': str,
                                   "TRANSACTION_AMT": float, 
                                   "SUB_ID": str
                                  }
                           )

In [4]:
contributions['zipcode_5'] = contributions['ZIP_CODE'].str[0:5]
contributions['ZIP_CODE'] = pd.to_numeric(contributions['ZIP_CODE'], errors='coerce')

In [5]:
contributions.dropna(subset=['CMTE_ID', 'ZIP_CODE','TRANSACTION_AMT','SUB_ID','zipcode_5'],inplace=True)
contributions = contributions.loc[contributions['TRANSACTION_AMT'] > 0]

In [6]:
# Illinois Exclusive

contributions = contributions.loc[(contributions['ZIP_CODE'] > 600000000) & (contributions['ZIP_CODE'] < 630000000)]

In [7]:
contributions.head()

,CMTE_ID,TRANSACTION_PGI,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,SUB_ID,zipcode_5
649,C00235739,P,603021617.0,02052015,208.0,4032020151240886655,60302
650,C00235739,P,603021617.0,02202015,208.0,4032020151240886656,60302
673,C00235739,P,601262235.0,02202015,270.0,4032020151240886522,60126
758,C00235739,NaN,604481479.0,02052015,139.0,4032020151240886209,60448
759,C00235739,NaN,604481479.0,02202015,145.0,4032020151240886210,60448


In [8]:
candidate_committee = pd.read_csv('candidate_committee.csv', sep='|')
candidate_committee = candidate_committee[['CAND_ID', 'CMTE_ID']]
candidate_committee.head()

,CAND_ID,CMTE_ID
0,H0AK00097,C00525261
1,H0AL02087,C00462143
2,H0AL02087,C00493783
3,H0AL05049,C00239038
4,H0AL05163,C00464149


In [9]:
candidates = pd.read_csv('candidates.csv', sep='|', names=['CAND_ID', 'CAND_PARTY'], usecols=[0, 2],).iloc[1:, :]
#candidates = candidates[['CAND_ID', 'CMTE_ID']]
candidates.head()
contributions = contributions.merge(candidate_committee.merge(candidates, how='inner'))
contributions.head()

,CMTE_ID,TRANSACTION_PGI,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,SUB_ID,zipcode_5,CAND_ID,CAND_PARTY
0,C00498667,P,607123716.0,01262015,2600.0,4041520151241868216,60712,H2CA43245,DEM
1,C00498667,G,607123716.0,01262015,2600.0,4041520151241868217,60712,H2CA43245,DEM
2,C00498667,P,606052715.0,03122015,1000.0,4041520151241868281,60605,H2CA43245,DEM
3,C00498667,P,606132919.0,11302015,100.0,4020320161261520742,60613,H2CA43245,DEM
4,C00498667,P,600355034.0,12172015,500.0,4020320161261520769,60035,H2CA43245,DEM


In [10]:
contributions.to_csv('contributions.csv', index=False)

In [11]:
grouped = contributions.groupby(['zipcode_5', 'CAND_PARTY'])

In [31]:
zip_summary_df = pd.DataFrame()
zip_summary_df['donations_sum'] = grouped['TRANSACTION_AMT'].sum()
zip_summary_df['donations_median'] = grouped['TRANSACTION_AMT'].median()
zip_summary_df['donations_count'] = grouped['SUB_ID'].count()
zip_summary_df.reset_index(inplace=True)
zip_summary_df.head()

,zipcode_5,CAND_PARTY,donations_sum,donations_median,donations_count
0,60002,DEM,10779.0,27.0,163
1,60002,REP,11510.0,25.0,181
2,60004,DEM,104788.0,30.0,1079
3,60004,REP,67493.0,100.0,421
4,60004,UNK,1000.0,500.0,2


In [32]:
zip_summary_df.dtypes

zipcode_5            object
CAND_PARTY           object
donations_sum       float64
donations_median    float64
donations_count       int64
dtype: object

In [41]:
zip_summary_df.to_csv('zipcode_donations.csv', index=True, index_label='id')

In [4]:
census_df = pd.read_csv('census_data_2016.csv',
                       usecols=['GEO.id', 'HD01_VD01', 'HD01_VD02',
                                'HC01_VC06', 'HC01_VC07','HC01_VC85', 'HC01_VC131',
                                'HC01_EST_VC08', 'HC01_EST_VC11', 'HC01_EST_VC13',
                                'HC01_EST_VC14', 'HC01_EST_VC15'])
header=['zipcode', 'pop_total', 'pop_white', 
        'pop_employed','pop_unemployed', 'median_household_income', 'pop_with_healthcare',
        'edu_25+_total', '25+_HS', 'edu_25+_assoc', 
        'edu_25+_bachelor', 'edu_25+_grad']
census_df = census_df[1:]
census_df.columns = header
census_df.head()
#https://www.census.gov/glossary/#term_Employed

C:\Users\alexa\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,4,16,17,108,155,158,160,161,162) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,zipcode,pop_total,pop_white,pop_employed,pop_unemployed,median_household_income,pop_with_healthcare,edu_25+_total,25+_HS,edu_25+_assoc,edu_25+_bachelor,edu_25+_grad
1,8600000US00601,17800,14436,3904,2152,11507,16503,11887,2985,604,1845,376
2,8600000US00602,39716,22941,11560,3116,15511,37497,27546,6076,3689,3840,1736
3,8600000US00603,51565,35176,12722,3768,16681,47081,35589,9590,2890,5809,2239
4,8600000US00606,6320,3739,1467,205,11648,6167,4381,1553,206,349,77
5,8600000US00610,27976,16259,8327,1587,17751,27106,19237,6329,1618,2368,626


In [5]:
for column in header[1:]:
    census_df[column] = pd.to_numeric(census_df[column], errors='coerce')
census_df.dtypes

zipcode                     object
pop_total                    int64
pop_white                    int64
pop_employed                 int64
pop_unemployed               int64
median_household_income    float64
pop_with_healthcare          int64
edu_25+_total                int64
25+_HS                       int64
edu_25+_assoc                int64
edu_25+_bachelor             int64
edu_25+_grad                 int64
dtype: object

In [6]:
census_summary_df = pd.DataFrame()
census_summary_df['zipcode_5'] = census_df['zipcode'].str[9:14]
census_summary_df['pop_total'] = census_df['pop_total']
census_summary_df['unemployment_rate'] = census_df['pop_unemployed'] / (census_df['pop_unemployed'] + census_df['pop_employed'])
census_summary_df['median_household_income'] = census_df['median_household_income']
census_summary_df['healthcare_rate'] = census_df['pop_with_healthcare'] / census_df['pop_total']
census_summary_df['hs_graduation_rate'] = census_df['25+_HS'] / census_df['edu_25+_total']
census_summary_df['assoc_degree_rate'] = census_df['edu_25+_assoc'] / census_df['edu_25+_total']
census_summary_df['bachelor_degree_rate'] = census_df['edu_25+_bachelor'] / census_df['edu_25+_total']
census_summary_df['grad_degree_rate'] = census_df['edu_25+_grad'] / census_df['edu_25+_total']
census_summary_df.head()

,zipcode_5,pop_total,unemployment_rate,median_household_income,healthcare_rate,hs_graduation_rate,assoc_degree_rate,bachelor_degree_rate,grad_degree_rate
1,00601,17800,0.355350,11507.0,0.927135,0.251115,0.050812,0.155212,0.031631
2,00602,39716,0.212319,15511.0,0.944128,0.220576,0.133921,0.139403,0.063022
3,00603,51565,0.228502,16681.0,0.913042,0.269465,0.081205,0.163225,0.062913
4,00606,6320,0.122608,11648.0,0.975791,0.354485,0.047021,0.079662,0.017576
5,00610,27976,0.160077,17751.0,0.968902,0.329001,0.084109,0.123096,0.032541


In [7]:
census_summary_df.dtypes
# Illinois Exclusive

census_summary_df = census_summary_df.loc[pd.to_numeric(census_summary_df['zipcode_5']) >= 60000].loc[pd.to_numeric(census_summary_df['zipcode_5']) < 63000]

In [8]:
census_summary_df.to_csv('census_data.csv', index=True, index_label='id')

In [9]:
census_summary_df.shape

(1383, 9)